In [1]:
import nltk
#nltk.download()
import os
import pandas as pd
# import numpy
# from os import listdir
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
#import matplotlib.pyplot as plt
import csv

In [2]:
######################################################## download data from the website
# import requests
# data_url = "https://www.kaggle.com/crowdflower/twitter-user-gender-classification/downloads/gender-classifier-DFE-791531.csv"
# kaggle_info = {'UserName': "avevanes", 'Password': "zqPwpz0t?"}
# r = requests.get(data_url)
# r = requests.post(r.url, data=kaggle_info)
# f = open('gender-classifier-DFE-791531.csv','w')
# for chunk in r.iter_content(chunk_size=512 * 1024):
#     if chunk:
#         f.write(chunk)
# f.close()



In [3]:
# clean
stop_words = stopwords.words('english')
def cleanAndNormalizeText(data):
    # tokenize
    tokenize = word_tokenize(data)
    # remove stop words
    filterText = [w for w in tokenize if w not in stop_words]
    filterText = [w for w in filterText if not len(w) <= 1]

    # stem
    ps = PorterStemmer()
    for i in range(len(filterText)-1):
        if len(filterText[i]) > 1:
            try:
                filterText[i] = ps.stem(filterText[i])
            except Exception as e:
                filterText[i] = filterText[i]

    return ' '.join(filterText)

In [4]:
# get data from csv
# check if need to pre-process description too

genderClassData = []
with open('C:\\Users\\abalata\\Downloads\\gender-classifier-DFE-791531.csv\\gender-classifier-DFE-791531.csv','r', encoding="utf8" ,errors="ignore") as csvfile:
    reader = csv.reader(csvfile)
    count = 0
    for row in reader:
        if count != 0:
            description = cleanAndNormalizeText(row[10])
            text = cleanAndNormalizeText(row[19])
            genderClassData.append([row[0], row[5], description, text])
        count = count+1

trainTable = pd.DataFrame(genderClassData, columns=['ID', 'gender', 'description', 'text'])
print(trainTable)


              ID   gender                                        description  \
0      815719226     male                                        sing rhythm   
1      815719227     male          'm author novel fill famili drama romance   
2      815719228     male                               loui whine squealing   
3      815719229     male  mobil guy 49er shazam googl kleiner perkin yah...   
4      815719230   female  ricki wilson the best frontman/kais chief the ...   
5      815719231   female                                           n't know   
6      815719232    brand  global marketplac imag video music share photo...   
7      815719233     male                   the secret get ahead get started   
8      815719234   female                  pll fan // crazi mcd // ramen bae   
9      815719235   female  renaiss art historian univers nottingham fuell...   
10     815719236    brand  clean food tast great provid energi nutrient N...   
11     815719237    brand               